In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import snntorch as snn
from snntorch import spikegen
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns

root_folder = "./tactile_dataset/"

velocities = [30]

num_textures = 12
texture_names = [f"texture_{i:02d}" for i in range(1, num_textures + 1)]

def load_and_merge_data(root, velocity, texture):
    velocity_folder = os.path.join(root, f"pickles_{velocity}")
    texture_folder = os.path.join(velocity_folder, texture)
    
    baro_file = os.path.join(texture_folder, "full_baro.csv")
    imu_file = os.path.join(texture_folder, "full_imu.csv")
    
    baro_df = pd.read_csv(baro_file)
    
    imu_df = pd.read_csv(imu_file)
    
    imu_df['baro'] = baro_df['baro']
        
    return imu_df



In [34]:

df_list = []

# for velocity in velocities:
#     for texture in texture_names:
#         df = load_and_merge_data(root_folder, velocity, texture).iloc[0:10000]
#         if df.empty:
#             print(f"No data for Velocity: {velocity} mm/s, Texture: {texture}. Skipping...")
#             continue
#         df_list.append(df)

# merged_df = pd.concat(df_list)

sliding_window_size=500
for velocity in velocities:
    for texture in texture_names:
        merged_df = load_and_merge_data(root_folder, velocity, texture)
        if merged_df.empty:
            print(f"No data for Velocity: {velocity} mm/s, Texture: {texture}. Skipping...")
            continue

        total_samples = len(merged_df)

        num_chunks = total_samples // sliding_window_size

        truncated_df = merged_df.iloc[:num_chunks * sliding_window_size]

        averaged_df = truncated_df.values.reshape(num_chunks, sliding_window_size, merged_df.shape[1]).mean(axis=1)

        averaged_df = pd.DataFrame(averaged_df, columns=merged_df.columns)

        df_list.append(averaged_df)

final_merged_df = pd.concat(df_list, ignore_index=True)



In [35]:
X = final_merged_df[['baro','imu_ax', 'imu_ay', 'imu_az','imu_gx', 'imu_gy', 'imu_gz','imu_mx', 'imu_my', 'imu_mz']].values
y = final_merged_df['Texture']

In [36]:

print(f"Total samples collected: {X.shape[0]}")
print(f"Feature vector size: {X.shape[1]}")
print(f"Unique textures: {np.unique(y)}")

Total samples collected: 90665
Feature vector size: 10
Unique textures: [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]


In [37]:

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
texture_classes = label_encoder.classes_
num_classes = len(texture_classes)
print("\nEncoded texture labels:", texture_classes)

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


Encoded texture labels: [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]


In [42]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# def count_classes(y, label_encoder):
#     unique, counts = np.unique(y, return_counts=True)
#     class_counts = dict(zip(label_encoder.inverse_transform(unique), counts))
#     return class_counts

# train_class_counts = count_classes(y_train, label_encoder)
# test_class_counts = count_classes(y_test, label_encoder)

# print("Training Set Class Distribution:")
# print(train_class_counts)
# print("\nTest Set Class Distribution:")
# print(test_class_counts)

def poisson_encoding(features, num_steps, device='cpu'):

    features_repeated = features.unsqueeze(1).repeat(1, num_steps, 1)
    
    rand_vals = torch.rand_like(features_repeated, device=device)
    
    spikes = (rand_vals < features_repeated).float()
    
    return spikes

def encode_to_spike_train(features, num_steps, device='cpu'):
    features_tensor = torch.tensor(features, dtype=torch.float32, device=device)

    spike_trains = poisson_encoding(features_tensor, num_steps=num_steps, device=device)
    
    return spike_trains

num_steps = 100
train_spike_trains = encode_to_spike_train(X_train, num_steps)
test_spike_trains = encode_to_spike_train(X_test, num_steps)

y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

batch_size = 64

train_dataset = TensorDataset(train_spike_trains, y_train_tensor)
test_dataset = TensorDataset(test_spike_trains, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [43]:


num_inputs = X_train.shape[1]
num_hidden = 100
num_outputs = num_classes
beta = 0.9

class SNNModel(nn.Module):
    def __init__(self):
        super(SNNModel, self).__init__()
        self.fc1 = nn.Linear(num_inputs, num_hidden)
        self.lif1 = snn.Leaky(beta=beta)
        self.fc2 = nn.Linear(num_hidden, num_hidden)
        self.lif2 = snn.Leaky(beta=beta)
        self.fc3 = nn.Linear(num_hidden, num_outputs)
        self.lif3 = snn.Leaky(beta=beta)

    def forward(self, x):
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        mem3 = self.lif3.init_leaky()

        spk_rec = []
        for step in range(x.size(1)):
            input_step = x[:, step, :]
            cur1 = self.fc1(input_step)
            spk1, mem1 = self.lif1(cur1, mem1)
            cur2 = self.fc2(spk1)
            spk2, mem2 = self.lif2(cur2, mem2)
            cur3 = self.fc3(spk2)
            spk3, mem3 = self.lif3(cur3, mem3)
            spk_rec.append(spk3)
        
        out_spikes = torch.stack(spk_rec, dim=0)
        return out_spikes

net = SNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

def forward_pass(net, data):

    spk_rec = net(data)
    return spk_rec

def train_model(net, train_loader, optimizer, criterion, num_epochs=20):

    net.train()
    for epoch in range(num_epochs):
        avg_loss = 0
        total_correct = 0
        total_samples = 0
        for data, targets in train_loader:
            data = data.float()
            targets = targets.long()
            spk_rec = forward_pass(net, data)
            spk_sum = spk_rec.sum(dim=0)
            loss = criterion(spk_sum, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            _, predicted = spk_sum.max(1)
            total_correct += (predicted == targets).sum().item()
            total_samples += targets.size(0)
        acc = total_correct / total_samples
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss / len(train_loader):.4f}, Accuracy: {acc*100:.2f}%")

def evaluate_model(net, test_loader):
    net.eval()
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for data, targets in test_loader:
            data = data.float()
            targets = targets.long()
            spk_rec = forward_pass(net, data)
            spk_sum = spk_rec.sum(dim=0)
            _, predicted = spk_sum.max(1)
            total_correct += (predicted == targets).sum().item()
            total_samples += targets.size(0)
    acc = total_correct / total_samples
    print(f"Test Accuracy: {acc*100:.2f}%")

num_epochs = 20
train_model(net, train_loader, optimizer, criterion, num_epochs)

evaluate_model(net, test_loader)

Epoch 1/20, Loss: 1.8571, Accuracy: 32.21%
Epoch 2/20, Loss: 1.6680, Accuracy: 36.94%
Epoch 3/20, Loss: 1.6356, Accuracy: 37.90%
Epoch 4/20, Loss: 1.6165, Accuracy: 38.49%
Epoch 5/20, Loss: 1.5998, Accuracy: 38.97%
Epoch 6/20, Loss: 1.5868, Accuracy: 39.02%
Epoch 7/20, Loss: 1.5771, Accuracy: 39.49%
Epoch 8/20, Loss: 1.5710, Accuracy: 39.84%
Epoch 9/20, Loss: 1.5636, Accuracy: 39.99%
Epoch 10/20, Loss: 1.5597, Accuracy: 40.24%
Epoch 11/20, Loss: 1.5536, Accuracy: 40.62%
Epoch 12/20, Loss: 1.5493, Accuracy: 40.47%
Epoch 13/20, Loss: 1.5458, Accuracy: 40.54%
Epoch 14/20, Loss: 1.5418, Accuracy: 40.66%
Epoch 15/20, Loss: 1.5389, Accuracy: 40.83%
Epoch 16/20, Loss: 1.5351, Accuracy: 41.06%
